In [98]:
import numpy as np
import pandas as pd

In [99]:
ANNOTATIONS_DIR = '../data/annotations/annotations averaged per song/song_level'
MFCC_DIR = '../data/mfcc'
METADATA_DIR = '../data/metadata'

df = pd.read_csv(f"{ANNOTATIONS_DIR}/static_annotations_averaged_songs_1_2000.csv", index_col='song_id')
print(df.shape)
df2 = pd.read_csv(f"{ANNOTATIONS_DIR}/static_annotations_averaged_songs_2000_2058.csv", index_col='song_id')
print(df2.shape)
for column in df2.columns.values:
   if column not in df.columns.values:
       df2 = df2.drop(column, axis = 1)
print(df2.shape)
df = df.append(df2)
print(df.values.shape)
song_ids = df.index.tolist()
print(len(song_ids))

(1744, 4)
(58, 12)
(58, 4)
(1802, 4)
1802


In [100]:
##Find Metadata Similarity
def createdic():
    df = pd.read_csv(f"{ANNOTATIONS_DIR}/static_annotations_averaged_songs_1_2000.csv", index_col='song_id')
    print(df.shape)
    df2 = pd.read_csv(f"{ANNOTATIONS_DIR}/static_annotations_averaged_songs_2000_2058.csv", index_col='song_id')
    print(df2.shape)
    for column in df2.columns.values:
        if column not in df.columns.values:
            df2 = df2.drop(column, axis = 1)
    print(df2.shape)
    df = df.append(df2)
    print(df.values.shape)
    song_ids = df.index.tolist()
    print(len(song_ids))
    df_meta1=pd.read_csv(f"{METADATA_DIR}/metadata_2013.csv")
    df_meta2=pd.read_csv(f"{METADATA_DIR}/metadata_2014.csv")
    df_meta3=pd.read_csv(f"{METADATA_DIR}/metadata_2015_2.csv")
    dic={}
    id_col1=df_meta1.columns[0]
    id_col2=df_meta2.columns[0]
    id_col3=df_meta3.columns[0]
    for song_id in song_ids:
        if song_id<=1000:
            str1=''
            for x in df_meta1.loc[df_meta1[id_col1]==song_id].values[0]:
                str1+=str(x)
            str1 = str1.replace('nan', ' ')
            str1 = str1.replace('.mp3', '')
            str1 = str1.replace(str(song_id), ' ')
            dic[song_id]=str1.replace('\t', ' ')
        elif song_id<=2000:
            str1=''
            for x in df_meta2.loc[df_meta2[id_col2]==song_id].values[0]:
                if(str(x) == 'nan'): continue
                str1+=str(x)
            str1 = str1.replace('nan', ' ')
            str1 = str1.replace('.mp3', '')
            str1 = str1.replace(str(song_id), ' ')
            dic[song_id]=str1.replace('\t', ' ')
        else:
            str1=''
            for x in df_meta3.loc[df_meta3[id_col3]==song_id].values[0]:
                if(str(x) == 'nan'): continue
                str1+=str(x)
            str1 = str1.replace('nan', ' ')
            str1 = str1.replace('.mp3', '')
            str1 = str1.replace(str(song_id), ' ')
            dic[song_id]=str1.replace('\t', ' ')
    return dic
song_metadic=createdic()
#print(song_metadic)

from difflib import SequenceMatcher
def find_metadata_similarity(song_id):
    metascore=[]
    for key in song_metadic.keys():
        if(key == song_id): continue
        metascore.append([key, SequenceMatcher(None, song_metadic[key], song_metadic[song_id]).ratio()])
    return metascore
print(find_metadata_similarity(2))

(1744, 4)
(58, 12)
(58, 4)
(1802, 4)
1802
[[3, 0.42735042735042733], [4, 0.42735042735042733], [5, 0.4666666666666667], [7, 0.4247787610619469], [8, 0.47058823529411764], [10, 0.2601626016260163], [12, 0.2956521739130435], [13, 0.4406779661016949], [17, 0.3937007874015748], [18, 0.33962264150943394], [19, 0.43859649122807015], [20, 0.21935483870967742], [21, 0.3220338983050847], [22, 0.7591240875912408], [24, 0.26153846153846155], [25, 0.291970802919708], [31, 0.37606837606837606], [32, 0.5483870967741935], [35, 0.44642857142857145], [37, 0.3076923076923077], [39, 0.3380281690140845], [40, 0.4], [41, 0.7], [42, 0.7083333333333334], [43, 0.4406779661016949], [44, 0.4098360655737705], [45, 0.40336134453781514], [46, 0.3968253968253968], [47, 0.39344262295081966], [48, 0.7101449275362319], [49, 0.5373134328358209], [50, 0.4098360655737705], [51, 0.47244094488188976], [52, 0.48484848484848486], [53, 0.6172839506172839], [54, 0.41935483870967744], [55, 0.36923076923076925], [56, 0.379310344

In [101]:
##Find Emotion Similarity
def find_emotion_similarity(song_id):
    this_song = np.array([df.loc[song_id].values[0], df.loc[song_id].values[2]])
    sim_scores = []
    for song in song_ids:
        if(song_id == song): continue
        curr_song = np.array([df.loc[song].values[0], df.loc[song].values[2]])
        dist = np.linalg.norm(this_song-curr_song)
        sim_scores.append([song, dist])
    return sim_scores

In [102]:
##Find Combined Similarity
def similarity(song_id):
    metadata_sim = np.array(find_metadata_similarity(song_id))
    emotion_sim = np.array(find_emotion_similarity(song_id))
    MAX_EMOTION = max(emotion_sim[:, 1])
    MIN_EMOTION = min(emotion_sim[:, 1])
    emotion_sim[:, 1] = (emotion_sim[:, 1] - MIN_EMOTION)/(MAX_EMOTION - MIN_EMOTION)
    MAX_METSCORE = max(metadata_sim[:, 1])
    MIN_METSCORE = min(metadata_sim[:, 1])
    metadata_sim[:, 1] = (metadata_sim[:, 1] - MIN_METSCORE)/(MAX_METSCORE - MIN_METSCORE)
    combined = {}
    for song in song_ids:
        if(song == song_id): continue
        metadata_score = metadata_sim[metadata_sim[:, 0] == song][0,1]
        emotion_score = emotion_sim[emotion_sim[:, 0] == song][0,1]
        combined_score = 0.5*(1-emotion_score) + 0.5*(metadata_score)
        combined[int(song)] = combined_score
    return(np.array(sorted(combined.items(), key=lambda x: x[1], reverse=True))[:10, 0])

In [103]:
print(similarity(3))

[  8. 168. 149. 209. 736. 579.  32. 712. 574.  76.]
